# Thoughts:
> 1. what to do about NANs and NULLs?  Set to 'UNK'?  
>> From ChatGPT 'In Transformers, "UNK" is a special token that represents an out-of-vocabulary (OOV) word, meaning a word that is not present in the vocabulary used by the language model. When a word is encountered during tokenization that is not present in the vocabulary, it is replaced with the UNK token.' <br>

> 2. details ...

***

In [1]:
import os
import glob
import pandas as pd
import numpy as np

# get the equity HF data (AAPL, JPM)
path = 'allfiles'
all_files = glob.glob(os.path.join(path,"**/*.csv.gz"))

np_array_list = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0) # read the csv's
    np_array_list.append(df.values) # convert to numpy array

# Create a bit array, just in case an array is useful later
comb_np_array = np.vstack(np_array_list)

# Create dataframe, for visualization, etc.
all_data_df = pd.DataFrame(comb_np_array)

print(all_data_df.shape)
all_data_df.head().T

(1070871, 61)


,0,1,2,3,4
0,20191218,20191218,20191218,20191218,20191218
1,JPM,JPM,JPM,JPM,JPM
2,04:00,04:01,04:02,04:03,04:04
3,04:00:00.000000000,04:01:00.000000000,04:02:00.000000000,04:03:00.000000000,04:04:00.000000000
4,NaN,137.28,137.29,137.91,137.9
...,...,...,...,...,...
56,0,0,0,0,0
57,NaN,NaN,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN
59,137.2335,137.27511,137.72427,137.80048,137.8247


In [2]:
# Set column headings
all_data_df.columns = ["Date","Ticker","TimeBarStart","OpenBarTime","OpenBidPrice","OpenBidSize", "OpenAskPrice","OpenAskSize","FirstTradeTime","FirstTradePrice","FirstTradeSize","HighBidTime","HighBidPrice","HighBidSize","HighAskTime","HighAskPrice","HighAskSize","HighTradeTime","HighTradePrice","HighTradeSize","LowBidTime","LowBidPrice","LowBidSize","LowAskTime","LowAskPrice","LowAskSize","LowTradeTime","LowTradePrice","LowTradeSize","CloseBarTime","CloseBidPrice","CloseBidSize","CloseAskPrice","CloseAskSize","LastTradeTime","LastTradePrice","LastTradeSize","MinSpread","MaxSpread","CancelSize","VolumeWeightPrice","NBBOQuoteCount","TradeAtBid","TradeAtBidMid","TradeAtMid","TradeAtMidAsk","TradeAtAsk","TradeAtCrossOrLocked","Volume","TotalTrades","FinraVolume","FinraVolumeWeightPrice","UptickVolume","DowntickVolume","RepeatUptickVolume","RepeatDowntickVolume","UnknownTickVolume","TradeToMidVolWeight","TradeToMidVolWeightRelative","TimeWeightBid","TimeWeightAsk"]

print(all_data_df.shape)
all_data_df.head().T

(1070871, 61)


,0,1,2,3,4
Date,20191218,20191218,20191218,20191218,20191218
Ticker,JPM,JPM,JPM,JPM,JPM
TimeBarStart,04:00,04:01,04:02,04:03,04:04
OpenBarTime,04:00:00.000000000,04:01:00.000000000,04:02:00.000000000,04:03:00.000000000,04:04:00.000000000
OpenBidPrice,NaN,137.28,137.29,137.91,137.9
...,...,...,...,...,...
UnknownTickVolume,0,0,0,0,0
TradeToMidVolWeight,NaN,NaN,NaN,NaN,NaN
TradeToMidVolWeightRelative,NaN,NaN,NaN,NaN,NaN
TimeWeightBid,137.2335,137.27511,137.72427,137.80048,137.8247


In [4]:
# Set a date-time index, using OpenBarTime
all_data_df['DateTimeIndex'] = pd.to_datetime(all_data_df['Date'].astype(str)) + pd.to_timedelta(all_data_df['OpenBarTime'].astype(str))
all_data_df = all_data_df.set_index('DateTimeIndex')

# Drop the original Date and TimeBarStart columns
all_data_df = all_data_df.drop(['Date','TimeBarStart'], axis=1)

print(all_data_df.shape)
all_data_df.head().T

(1070871, 59)


DateTimeIndex,2019-12-18 04:00:00,2019-12-18 04:01:00,2019-12-18 04:02:00,2019-12-18 04:03:00,2019-12-18 04:04:00
Ticker,JPM,JPM,JPM,JPM,JPM
OpenBarTime,04:00:00.000000000,04:01:00.000000000,04:02:00.000000000,04:03:00.000000000,04:04:00.000000000
OpenBidPrice,NaN,137.28,137.29,137.91,137.9
OpenBidSize,NaN,100.0,100.0,100.0,100.0
OpenAskPrice,NaN,138.51,138.54,138.52,138.56
OpenAskSize,NaN,100.0,100.0,100.0,100.0
FirstTradeTime,NaN,NaN,NaN,NaN,NaN
FirstTradePrice,NaN,NaN,NaN,NaN,NaN
FirstTradeSize,NaN,NaN,NaN,NaN,NaN
HighBidTime,04:00:23.259114982,04:01:41.139964270,04:02:46.789510918,04:03:23.830113141,04:04:59.130469187


In [8]:
# import HF datasets
import pandas as pd
df_data = pd.read_csv("data.csv", index_col=False, header=0, engine='python')

print(df_data.shape)
df_data.head(5).T

(1079514, 62)


,0,1,2,3,4
DateTimeIndex,2018-01-02 04:00:00,2018-01-02 04:01:00,2018-01-02 04:02:00,2018-01-02 04:03:00,2018-01-02 04:04:00
Date,20180102,20180102,20180102,20180102,20180102
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
TimeBarStart,04:00,04:01,04:02,04:03,04:04
OpenBarTime,04:00:00.000000000,04:01:00.000000000,04:02:00.000000000,04:03:00.000000000,04:04:00.000000000
...,...,...,...,...,...
UnknownTickVolume,18,0,0,0,0
TradeToMidVolWeight,-1.73183,NaN,NaN,1.27778,NaN
TradeToMidVolWeightRelative,0.22436,NaN,NaN,0.03194,NaN
TimeWeightBid,169.04928,169.05125,169.1,169.1,169.1


In [12]:
# drop rows with missing values
df_data = df_data.dropna()
print(df_data.shape)

(2879, 62)


In [ ]:
# Show datatype of columns
print(df_data.dtypes.to_string(max_rows=None))

In [ ]:
# find columns of type object
object_cols = df_data.select_dtypes(include='object').columns.tolist()
print(object_cols)

In [11]:
print(df_data['DateTimeIndex'].value_counts())

2018-01-02 04:00:00    2
2019-07-02 19:52:00    2
2019-07-02 19:50:00    2
2019-07-02 19:49:00    2
2019-07-02 19:48:00    2
                      ..
2019-03-07 05:33:00    1
2019-09-06 20:00:00    1
2019-10-30 20:00:00    1
2019-03-07 05:32:00    1
2020-03-31 20:00:00    1
Name: DateTimeIndex, Length: 540161, dtype: int64


In [10]:
# df_data['Ticker'] = df_data['Ticker'].astype(str)
# df_data['Ticker'] = df_data['Ticker'].astype(str, errors='coerce')
df_data['DateTimeIndex'] = pd.to_datetime(df_data['DateTimeIndex']).astype(str)
df_data.dtypes

DateTimeIndex                   object
Date                             int64
Ticker                          object
TimeBarStart                    object
OpenBarTime                     object
                                ...   
UnknownTickVolume                int64
TradeToMidVolWeight            float64
TradeToMidVolWeightRelative    float64
TimeWeightBid                  float64
TimeWeightAsk                  float64
Length: 62, dtype: object

In [ ]:
# Set datatypes for columns
df_data['DateTimeIndex'] = pd.to_datetime(df_data['DateTimeIndex']).astype(str)
df_data['Ticker'] = df_data['Ticker'].astype(str)
df_data['TimeBarStart'] = df_data['TimeBarStart'].astype(str)
df_data['OpenBarTime'] = df_data['OpenBarTime'].astype(str)
df_data['FirstTradeTime'] = df_data['FirstTradeTime'].astype(str)
df_data['HighBidTime'] = df_data['HighBidTime'].astype(str)
df_data['HighAskTime'] = df_data['HighAskTime'].astype(str)
df_data['HighTradeTime'] = df_data['HighTradeTime'].astype(str)
df_data['LowBidTime'] = df_data['LowBidTime'].astype(str)
df_data['LowAskTime'] = df_data['LowAskTime'].astype(str)
df_data['LowTradeTime'] = df_data['LowTradeTime'].astype(str)
df_data['CloseBarTime'] = df_data['CloseBarTime'].astype(str)
df_data['LastTradeTime'] = df_data['LastTradeTime'].astype(str)
df_data['CloseBidSize'] = df_data['CloseBidSize'].astype(float)
df_data['CloseAskSize'] = df_data['CloseAskSize'].astype(float)
df_data['CloseBidPrice'] = df_data['CloseBidPrice'].astype(float)
df_data['CloseAskPrice'] = df_data['CloseAskPrice'].astype(float)

# Show datatype of columns
print(df_data.dtypes.to_string(max_rows=None))

In [ ]:
# Show datatype of columns
print(df_data.dtypes.to_string(max_rows=None))

In [ ]:
# find columns of type object
object_cols = df_data.select_dtypes(include='object').columns.tolist()

# print the columns of type object
print(object_cols)

In [ ]:
# Reduce df_data to smF
smF = df_data[['Ticker','CloseBidSize','CloseAskSize','CloseBidPrice',
                'CloseAskPrice']].copy()   
smF['Date'] = pd.to_datetime(df_data['Date'].astype(str))
smF['Time'] = pd.to_timedelta(df_data['OpenBarTime'].astype(str))
smF['DateTime'] = pd.to_datetime(df_data['Date'].astype(str)) + pd.to_timedelta(df_data['OpenBarTime'].astype(str))

In [ ]:
# Compute WeightedMidPrice using the closing prices per analysis
smF['WeightedMidPrice'] = ((smF['CloseBidSize']*smF['CloseAskPrice']) + (smF['CloseAskSize']*smF['CloseBidPrice'])) / (smF['CloseBidSize'] + smF['CloseAskSize'])


In [ ]:
# Raw returns
AAPL_rr = smF.loc[smF['Ticker'] == "AAPL"]
AAPL_rr = AAPL_rr['WeightedMidPrice'] - AAPL_rr['WeightedMidPrice'].shift(1)
AAPL_rr = AAPL_rr[AAPL_rr.notna()].copy()
AAPL_rr = AAPL_rr[AAPL_rr != 0].copy()
JPM_rr = smF.loc[smF['Ticker'] == "JPM"]
JPM_rr = JPM_rr['WeightedMidPrice'] - JPM_rr['WeightedMidPrice'].shift(1)
JPM_rr = JPM_rr[JPM_rr.notna()].copy()
JPM_rr = JPM_rr[JPM_rr != 0].copy()
    
# Log returns
AAPL_lr = smF.loc[smF['Ticker'] == "AAPL"]
AAPL_lr = np.log(AAPL_lr['WeightedMidPrice'].astype(float))
AAPL_lr = AAPL_lr - AAPL_lr.shift(1)
AAPL_lr = AAPL_lr[AAPL_lr.notna()].copy()
AAPL_lr = AAPL_lr[AAPL_lr != 0].copy()
JPM_lr = smF.loc[smF['Ticker'] == "JPM"]
JPM_lr = np.log(JPM_lr['WeightedMidPrice'].astype(float))
JPM_lr = JPM_lr - JPM_lr.shift(1)
JPM_lr = JPM_lr[JPM_lr.notna()].copy()
JPM_lr = JPM_lr[JPM_lr != 0].copy()

In [ ]:
# Remove outliers
Q1l = AAPL_lr.quantile(0.001)   
Q3l = AAPL_lr.quantile(0.999)   
IQl = Q3l - Q1l
Q1r = AAPL_rr.quantile(0.001)   
Q3r = AAPL_rr.quantile(0.999)   
IQr = Q3r - Q1r
AAPL_lr = AAPL_lr[~((AAPL_lr < (Q1l - 1.5 * IQl)) | (AAPL_lr > (Q3l + 1.5 * IQl)))]
AAPL_rr = AAPL_rr[~((AAPL_rr < (Q1r - 1.5 * IQr)) | (AAPL_rr > (Q3r + 1.5 * IQr)))]
JPM_lr = JPM_lr[~((JPM_lr < (Q1l - 1.5 * IQl)) | (JPM_lr > (Q3l + 1.5 * IQl)))]
JPM_rr = JPM_rr[~((JPM_rr < (Q1r - 1.5 * IQr)) | (JPM_rr > (Q3r + 1.5 * IQr)))]
    
# log returns only for models split into estimate (E=60%) and out-of-forecast (F=40%)
AAPL = AAPL_lr.to_numpy(copy=True)
JPM = JPM_lr.to_numpy(copy=True)
aaplE = AAPL[0:269618,]
aaplF = AAPL[269618:,]
jpmE = JPM[0:200363,]
jpmF = JPM[200363:,]
aaplE = aaplE[:,np.newaxis]
aaplF = aaplF[:,np.newaxis]
jpmE = jpmE[:,np.newaxis]
jpmF = jpmF[:,np.newaxis]

print(
    df_data.shape,
    smF.shape,
    AAPL_rr.shape, 
    JPM_rr.shape, 
    AAPL_lr.shape, 
    JPM_lr.shape, 
    aaplE.shape, 
    aaplF.shape, 
    jpmE.shape, 
    jpmF.shape
    )